In [ ]:
import glob
import re
import json
import itertools
import pandas as pd
from pathlib import Path
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import Lambda

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

from tqdm import tqdm

from sklearn.model_selection import GridSearchCV
import joblib
from sklearn.model_selection import TimeSeriesSplit

# all the functions from helpers.py
from helpers_scenario2_redes import *

In [ ]:
annotations_folder = '../data/raw/scenario_2/fold_0/train/annotations/'
# physiology_folder = "../data/preprocessed/cleaned/scenario_1/fold_0/train/physiology/" #'../data/raw/scenario_1/train/physiology/'
physiology_folder = "../data/preprocessed/cleaned_and_prepro_improved/scenario_2/fold_0/train/physiology/" #'../data/raw/scenario_1/train/physiology/'data\preprocessed\

df_physiology = load_read_and_append_csvs(physiology_folder)
df_annotations = load_read_and_append_csvs(annotations_folder)

videos = df_physiology.video.unique()
subjects = df_physiology.subject.unique()

splits = split_subjects_train_test(subjects, 3)

In [ ]:
#videos = [10]

In [ ]:
from keras.layers import Dropout
from keras.regularizers import l1
from keras.layers import Activation

def create_cnn_lstm_model(input_shape, lstm_units=64, dropout_rate=0.3, kernel_regularizer_l1=0.001):
    input_signal = Input(shape=input_shape)

    x = Conv1D(16, 5, padding='same')(input_signal)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropout_rate)(x)

    x = Conv1D(8, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropout_rate)(x)

    x = LSTM(lstm_units)(x)

    x = Dense(64, activation='relu', kernel_regularizer=l1(kernel_regularizer_l1))(x)

    return Model(inputs=input_signal, outputs=x)

In [ ]:
from concurrent.futures import ProcessPoolExecutor, as_completed, ThreadPoolExecutor
import threading
import multiprocessing
from tqdm import tqdm
import itertools
import json


# Initialize an empty dictionary to store the video results
video_results = {}

# Wrap the outer loop with tqdm
for video in tqdm(videos, desc="Processing videos", unit="video", bar_format="{l_bar}%s{bar}%s{r_bar}" % ('\033[32m', '\033[0m')):
    print(f"Processing video: {video}")

    df_physiology_video = df_physiology.loc[df_physiology.video == video]
    df_annotations_video = df_annotations.loc[df_annotations.video == video]

    rmses = []

    for split_index, split in enumerate(splits):
        # print(split)

        X_train, X_test, y_train, y_test, numeric_column_indices, categorical_column_indices = preprocess(
            df_physiology_video.copy(), df_annotations_video.copy(), split=split, predictions_cols=['arousal', 'valence'], aggregate=None,
            window_duration=10000, resample_rate=100)

        print("X_test shape:", X_test.shape)
        print("y_test shape:", y_test.shape)
                
        # Extract arousal and valence values from y_train and y_test
        y_arousal_train = y_train[:, 0]
        y_valence_train = y_train[:, 1]
        y_arousal_test = y_test[:, 0]
        y_valence_test = y_test[:, 1]

        input_shape = (X_train.shape[1], X_train.shape[2])
        cnn_model = create_cnn_lstm_model(input_shape)

        # Create separate heads for valence and arousal prediction, using sigmoid activation and scaling the output
        valence_output = Dense(1, activation='sigmoid')(cnn_model.output)
        valence_output = Lambda(lambda x: x * 8 + 1, name='valence_output')(valence_output)

        arousal_output = Dense(1, activation='sigmoid')(cnn_model.output)
        arousal_output = Lambda(lambda x: x * 8 + 1, name='arousal_output')(arousal_output)

        # Combine the model
        final_model = Model(inputs=cnn_model.input, outputs=[valence_output, arousal_output])

        # Compile the model
        final_model.compile(optimizer='adam',
                            loss={'valence_output': 'mse',
                                'arousal_output': 'mse'},
                            metrics=[tf.keras.metrics.RootMeanSquaredError()])

        # Set up early stopping
        early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

        # Train the model
        history = final_model.fit(X_train, {'valence_output': y_valence_train, 'arousal_output': y_arousal_train},
                                validation_split=0.2, epochs=50, batch_size=32,
                                callbacks=[early_stopping_callback])
        
        evaluation = final_model.evaluate(X_test, [y_valence_test, y_arousal_test])

        # Save the evaluation results in the dictionary
        key = f"video_{video}_split_{split_index}"
        results[key] = {
            'valence_output_root_mean_squared_error': evaluation[3],
            'arousal_output_root_mean_squared_error': evaluation[4]
        }

    # Calculate the average of the evaluation results
    evaluation_sum = np.array([list(rmse_dict.values()) for rmse_dict in results.values()]).sum(axis=0)
    average_evaluation = evaluation_sum / len(splits)

    # Save the results for the current video
    video_results[f"video_{video}"] = results

    # Save the average evaluation in the dictionary
    video_results[f"video_{video}_average"] = {
        'valence_output_root_mean_squared_error': average_evaluation[0],
        'arousal_output_root_mean_squared_error': average_evaluation[1]
    }

    # Reset the results dictionary for the next video
    results = {}

# Save the video_results dictionary to a JSON file
with open('results_separate_videos.json', 'w') as outfile:
    json.dump(video_results, outfile, indent=4)

# Print the final results
for video_key, video_value in video_results.items():
    print(f"{video_key}: {video_value}")

In [ ]:
print("Results for each split:")
print(results)

In [ ]:
print(final_model.evaluate(X_test, [y_valence_test, y_arousal_test]))

In [ ]:
print("X_train shape:", X_train.shape)
print("y_valence_train shape:", y_valence_train.shape)
print("y_arousal_train shape:", y_arousal_train.shape)
print("X_test shape:", X_test.shape)
print("y_valence_test shape:", y_valence_test.shape)
print("y_arousal_test shape:", y_arousal_test.shape)

In [ ]:
[split for split_index, split in enumerate(splits)]

In [ ]:
X_test = np.swapaxes(X_test, 0, 1)

In [ ]:
print(final_model.evaluate(X_test, [y_valence_test, y_arousal_test]))


In [ ]:
print(X_test.shape)
print(y_valence_test.shape)
print(y_arousal_test.shape)
